In [1]:
#!pip install import_ipynb
import import_ipynb
import matplotlib.pyplot as plt
from FULL_DATA import final_df

importing Jupyter notebook from FULL_DATA.ipynb
importing Jupyter notebook from Create_Labels.ipynb
(6383, 4)
4377
(4402, 7)
(4377, 7)
(4437, 7)


In [2]:
final_df['SENTIMENT'] = [0 if (x > 1 and x<3.4999) else 1 if (x > 3.5 and x<4.49999) else -1  for x in final_df['ratings']]
print(final_df['SENTIMENT'].value_counts())
print(final_df.shape)
final_df.head()

 1    2367
 0    1144
-1     866
Name: SENTIMENT, dtype: int64
(4377, 8)


,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT
14602,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1
14599,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1
14592,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1
14589,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1
14586,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1


In [3]:
import pandas as pd
#final_df = pd.read_csv('final_df_rev1.csv')

final_df['SENTIMENT'].value_counts()
#final_df["TRANSCRIPT_DATE"].min()
#final_df["TRANSCRIPT_DATE"].max()

 1    2367
 0    1144
-1     866
Name: SENTIMENT, dtype: int64

In [4]:
final_df.head()

,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT
14602,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1
14599,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1
14592,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1
14589,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1
14586,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1


In [5]:
### Multi-class text classification
from nltk.tokenize import regexp_tokenize
import string
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """
def tokenize(text, pattern = default_pattern):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return regexp_tokenize(text, pattern)
final_df['TOKENIZED_TRANSCRIPT'] = final_df.apply(lambda row: tokenize(row['TRANSCRIPTS']), axis=1)
final_df['TOKENIZED_TRANSCRIPT'] = final_df['TOKENIZED_TRANSCRIPT'].apply(lambda x: ' '.join(x))


In [6]:
#final_df.to_csv('final_df_tokenized.csv', index = False)


In [7]:
final_df_copy = final_df.copy()

In [8]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
counts = count_vect.fit_transform(final_df_copy['TOKENIZED_TRANSCRIPT'])

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, final_df_copy['SENTIMENT'], test_size=0.2, random_state=0)

clf = linear_model.LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
logreg_model = clf.fit(X_train, y_train)
y_pred_prob = logreg_model.predict_proba(X_test)

y_pred = np.argmax(y_pred_prob, axis = 1)
y_pred[y_pred == 0] = -1
y_pred[y_pred == 1] = 0
y_pred[y_pred == 2] = 1


logreg_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(logreg_results.head(10)) 

print(logreg_results['Predicted'].value_counts())

print('Multinomial Logistic Regression Accuracy: ', accuracy_score(y_test, y_pred))
print('Multinomial Logistic Regression F1 Score: ', f1_score(y_test, y_pred, average='macro'))

       Actual  Predicted
19522      -1          1
18997       1          1
7507        1          1
2173        1          1
11857       1          1
12629      -1          1
3370        1          1
11324       0          1
315        -1          1
6260        1          1
 1    779
 0     71
-1     26
Name: Predicted, dtype: int64
Multinomial Logistic Regression Accuracy:  0.6495433789954338
Multinomial Logistic Regression F1 Score:  0.48488863160112333


In [9]:
final_df_copy = final_df.copy()

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
counts = count_vect.fit_transform(final_df_copy['TOKENIZED_TRANSCRIPT'])

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, final_df_copy['SENTIMENT'], test_size=0.2, random_state=0)
priors = [y_train.value_counts()[-1]/len(y_train), y_train.value_counts()[0]/len(y_train), y_train.value_counts()[1]/len(y_train)]

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(class_prior=priors).fit(X_train, y_train)


import numpy as np
predicted = model.predict(X_test)

nb_results = pd.DataFrame({'Actual': y_test, 'Predicted': predicted})
print(nb_results.head(10)) 
print(nb_results['Predicted'].value_counts())

print('Multinomial Naive Bayes Accuracy: ', accuracy_score(y_test, predicted))
print('Multinomial Naive Bayes F1 Score: ', f1_score(nb_results['Actual'], nb_results['Predicted'], average='macro'))

       Actual  Predicted
19522      -1          1
18997       1          1
7507        1          1
2173        1          1
11857       1          1
12629      -1          1
3370        1          1
11324       0          1
315        -1          1
6260        1          1
1    876
Name: Predicted, dtype: int64
Multinomial Naive Bayes Accuracy:  0.5513698630136986
Multinomial Naive Bayes F1 Score:  0.23693892568064756


C:\Users\Frank\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
final_df_copy = final_df.copy()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=0)),])
X_train, X_test, y_train, y_test = train_test_split(final_df_copy['TOKENIZED_TRANSCRIPT'], final_df_copy['SENTIMENT'], test_size=0.2, random_state=0)
svm = text_clf_svm.fit(X_train, y_train)
predicted = text_clf_svm.predict(X_test)

svm_results = pd.DataFrame({'Actual': y_test, 'Predicted': predicted})
print(svm_results.head(10)) 
print(svm_results['Predicted'].value_counts())

print('SVM Accuracy: ', np.mean(predicted == y_test))
print('SVM F1 Score: ', f1_score(svm_results['Actual'], svm_results['Predicted'], average='macro'))

       Actual  Predicted
19522      -1          1
18997       1          1
7507        1          1
2173        1          1
11857       1          1
12629      -1          1
3370        1          1
11324       0          1
315        -1          1
6260        1          1
 1    874
-1      1
 0      1
Name: Predicted, dtype: int64
SVM Accuracy:  0.5536529680365296
SVM F1 Score:  0.24412015441154053


In [12]:
final_df_stem = final_df.copy()

In [13]:
final_df_stem['TOKENIZED_TRANSCRIPT'] = final_df_stem.apply(lambda row: tokenize(row['TRANSCRIPTS']), axis=1)
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
final_df_stem['TOKENIZED_TRANSCRIPT'] = final_df_stem['TOKENIZED_TRANSCRIPT'].apply(lambda x: [stemmer.stem(y) for y in x])


In [14]:
final_df_stem_copy = final_df_stem.copy()


In [15]:
#stemmed_final_df.to_csv('stemmed_final_df.csv')

In [16]:
#final_df['TOKENIZED_TRANSCRIPT'] = final_df['TOKENIZED_TRANSCRIPT'].apply(lambda x: ' '.join(x))

from sklearn import linear_model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
counts = count_vect.fit_transform(final_df['TOKENIZED_TRANSCRIPT'])

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, final_df_stem_copy['SENTIMENT'], test_size=0.2, random_state=0)

clf = linear_model.LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
logreg_model = clf.fit(X_train, y_train)
y_pred_prob = logreg_model.predict_proba(X_test)

y_pred = np.argmax(y_pred_prob, axis = 1)
y_pred[y_pred == 0] = -1
y_pred[y_pred == 1] = 0
y_pred[y_pred == 2] = 1


logreg_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(logreg_results.head(10)) 

correct = np.sum(y_pred == y_test)
print(logreg_results['Predicted'].value_counts())
print('Number correct:', str(correct))
print('Stemmed Multinomial Logistic Regression Accuracy: ', correct/len(y_pred))
print('Stemmed Multinomial Logistic Regression F1 Score: ', f1_score(logreg_results['Actual'], logreg_results['Predicted'], average='macro'))

       Actual  Predicted
19522      -1          1
18997       1          1
7507        1          1
2173        1          1
11857       1          1
12629      -1          1
3370        1          1
11324       0          1
315        -1          1
6260        1          1
 1    779
 0     71
-1     26
Name: Predicted, dtype: int64
Number correct: 569
Stemmed Multinomial Logistic Regression Accuracy:  0.6495433789954338
Stemmed Multinomial Logistic Regression F1 Score:  0.48488863160112333


In [17]:
final_df_stem_copy = final_df_stem.copy()
count_vect = CountVectorizer()
counts = count_vect.fit_transform(final_df['TOKENIZED_TRANSCRIPT'])

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer().fit(counts)
counts = transformer.transform(counts)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, final_df_stem_copy['SENTIMENT'], test_size=0.2, random_state=0)
priors = [y_train.value_counts()[-1]/len(y_train), y_train.value_counts()[0]/len(y_train), y_train.value_counts()[1]/len(y_train)]


from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(class_prior=priors).fit(X_train, y_train)

import numpy as np
predicted = model.predict(X_test)

nb_results = pd.DataFrame({'Actual': y_test, 'Predicted': predicted})
print(nb_results.head(10)) 
print(nb_results['Predicted'].value_counts())

print('Stemmed Multinomial Naive Bayes Accuracy: ', np.mean(predicted == y_test))
print('Stemmed Multinomial Naive Bayes F1 Score: ', f1_score(nb_results['Actual'], nb_results['Predicted'], average='macro'))

       Actual  Predicted
19522      -1          1
18997       1          1
7507        1          1
2173        1          1
11857       1          1
12629      -1          1
3370        1          1
11324       0          1
315        -1          1
6260        1          1
1    876
Name: Predicted, dtype: int64
Stemmed Multinomial Naive Bayes Accuracy:  0.5513698630136986
Stemmed Multinomial Naive Bayes F1 Score:  0.23693892568064756


C:\Users\Frank\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
final_df_stem

,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT,TOKENIZED_TRANSCRIPT
14602,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1,"[imag, sourc, the, motley, fool, agil, technol..."
14599,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1,"[imag, sourc, the, motley, fool, agil, technol..."
14592,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1,"[imag, sourc, the, motley, fool, agil, technol..."
14589,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1,"[imag, sourc, the, motley, fool, agil, technol..."
14586,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1,"[imag, sourc, the, motley, fool, agil, technol..."
14583,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-11-25,2019-12-31,4.067,36 days,A_2019-12-31,1,"[imag, sourc, the, motley, fool, agil, technol..."
15694,"['Image source: The Motley Fool.', 'Alcoa (NYS...",AA,2018-06-30,2018-09-28,4.294,90 days,AA_2018-09-28,1,"[imag, sourc, the, motley, fool, alcoa, nyseaa..."
15684,"['Image source: The Motley Fool.', 'Alcoa Corp...",AA,2019-01-16,2019-03-29,4.444,72 days,AA_2019-03-29,1,"[imag, sourc, the, motley, fool, alcoa, corpxa..."
15681,"['Image source: The Motley Fool.', 'ALCOA CORP...",AA,2019-04-17,2019-06-28,4.294,72 days,AA_2019-06-28,1,"[imag, sourc, the, motley, fool, alcoa, corpxa..."
15678,"['Image source: The Motley Fool.', 'Alcoa Corp...",AA,2019-07-17,2019-09-30,3.875,75 days,AA_2019-09-30,1,"[imag, sourc, the, motley, fool, alcoa, corpor..."


In [19]:
final_df_stem_copy = final_df_stem.copy()
final_df_stem_copy['TOKENIZED_TRANSCRIPT'] = final_df_stem_copy['TOKENIZED_TRANSCRIPT'].apply(lambda x: ' '.join(x))
final_df_stem_copy['TOKENIZED_TRANSCRIPT'] = final_df_stem_copy.apply(lambda row: tokenize(row['TRANSCRIPTS']), axis=1)
final_df_stem_copy['TOKENIZED_TRANSCRIPT'] = final_df_stem_copy['TOKENIZED_TRANSCRIPT'].apply(lambda x: ' '.join(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=0)),])
X_train, X_test, y_train, y_test = train_test_split(final_df_stem_copy['TOKENIZED_TRANSCRIPT'], final_df_stem_copy['SENTIMENT'], test_size=0.2, random_state=0)
svm = text_clf_svm.fit(X_train, y_train)
predicted = text_clf_svm.predict(X_test)

svm_results = pd.DataFrame({'Actual': y_test, 'Predicted': predicted})
print(svm_results.head(10)) 
print(svm_results['Predicted'].value_counts())

print('Stemmed SVM Accuracy: ', np.mean(predicted == y_test))
print('Stemmed SVM F1 Score: ', f1_score(svm_results['Actual'], svm_results['Predicted'], average='macro'))

       Actual  Predicted
19522      -1          1
18997       1          1
7507        1          1
2173        1          1
11857       1          1
12629      -1          1
3370        1          1
11324       0          1
315        -1          1
6260        1          1
 1    874
-1      1
 0      1
Name: Predicted, dtype: int64
Stemmed SVM Accuracy:  0.5536529680365296
Stemmed SVM F1 Score:  0.24412015441154053


In [20]:
final_df_stem_copy.head()

,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT,TOKENIZED_TRANSCRIPT
14602,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1,image source the motley fool agilent technolog...
14599,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1,image source the motley fool agilent technolog...
14592,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1,image source the motley fool agilent technolog...
14589,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1,image source the motley fool agilent technolog...
14586,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1,image source the motley fool agilent technolog...
